# Alpine Resort Shutdown Model

Implementation of the Seq2Seq LSTM model described in the technical specification. 
Goal: Predict shutdown year based on variable-length weather sequences.

**Note:** Using dummy data for now until we get the real CSV exports.

In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, LSTM, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam

# basic setup
tf.random.set_seed(42)
np.random.seed(42)

/Users/admin/Documents/KTH-Projects_Code/Year-5/P2/.venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# --- DATA SIMULATION ---
# we don't have the csv yet, so mocking the expected structure
# assuming 52 weekly temp readings per year -> 52 features/timestep

def generate_dummy_data(n_samples=100, max_years=15):
    # X: (samples, timesteps, features)
    # y: (samples, timesteps, 1)
    
    X_list = []
    y_list = []
    
    for _ in range(n_samples):
        # random seq length (5 to max_years)
        # tests that model handles variable years correctly
        n_years = np.random.randint(5, max_years + 1)
        
        # 52 weeks of temp data per year
        resort_weather = np.random.normal(0, 10, size=(n_years, 52))
        
        # determine if/when it shuts down
        # 30% chance it survives the whole period
        shutdown_year = np.random.randint(1, n_years) if np.random.rand() > 0.3 else -1
        
        labels = np.zeros((n_years, 1))
        
        if shutdown_year != -1:
            # ghost data logic: once it's dead, it stays dead
            # teaching model about the irreversible state
            labels[shutdown_year:] = 1.0
            
        X_list.append(resort_weather)
        y_list.append(labels)

    # padding is needed for batching, masking layer will ignore -999 later
    X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_list, padding='post', value=-999, dtype='float32')
    y_padded = tf.keras.preprocessing.sequence.pad_sequences(y_list, padding='post', value=-999, dtype='float32')
    
    return X_padded, y_padded

X_train, y_train = generate_dummy_data()
# sanity check shapes
print(f"X shape: {X_train.shape}") 
print(f"y shape: {y_train.shape}")

X shape: (100, 15, 52)
y shape: (100, 15, 1)


In [3]:
# --- MODEL DEFINITION ---

class ResortShutdownModel:
    def __init__(self, n_features=52):
        self.model = self._build_architecture(n_features)
        
    def _build_architecture(self, n_features):
        # input shape=(None, features) allows any sequence length
        inputs = Input(shape=(None, n_features))
        
        # tell layers to ignore padded values
        x = Masking(mask_value=-999)(inputs)
        
        # return_sequences=True is mandatory here
        # we need a prediction for EVERY year, not just the final state
        x = LSTM(64, return_sequences=True)(x)
        x = LSTM(32, return_sequences=True)(x)
        
        # independent classification per timestep
        outputs = TimeDistributed(Dense(1, activation='sigmoid'))(x)
        
        model = Model(inputs, outputs)
        
        # binary crossentropy fits the alive/dead (0/1) logic
        model.compile(optimizer=Adam(learning_rate=0.001), 
                     loss='binary_crossentropy', 
                     metrics=['accuracy'])
        return model

    def train(self, X, y, epochs=10, batch_size=16):
        print("Starting training loop...")
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.1)
        
    def predict_status(self, weather_sequence):
        # expecting single sequence: (N_Years, 52)
        # add batch dim for prediction
        probs = self.model.predict(np.expand_dims(weather_sequence, 0))[0]
        return probs.flatten()

# init model
shutdown_model = ResortShutdownModel(n_features=52)
shutdown_model.model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 52)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None, 52)  │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking (Masking)   │ (None, None, 52)  │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any (Any)           │ (None, None)      │          0 │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, None, 64)  │     29,952 │ masking[0][0],    │
│                     │                   │            │ any[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, None, 32)  │     12,416 │ lstm[0][0],       │
│                     │                   │            │ any[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, None, 1)   │         33 │ lstm_1[0][0],     │
│ (TimeDistributed)   │                   │            │ any[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 42,401 (165.63 KB)

 Trainable params: 42,401 (165.63 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# --- TRAINING CHECK ---
# just verifying that loss decreases
shutdown_model.train(X_train, y_train, epochs=5)

Starting training loop...
Epoch 1/5



1/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5917 - loss: 0.6797


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.5461 - loss: 0.6877 - val_accuracy: 0.4302 - val_loss: 0.6981


Epoch 2/5



1/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7988 - loss: 0.6129

...


1/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8107 - loss: 0.5448


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8719 - loss: 0.5473 - val_accuracy: 0.4651 - val_loss: 0.7132


Epoch 5/5



1/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8284 - loss: 0.5044


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9012 - loss: 0.4903 - val_accuracy: 0.4767 - val_loss: 0.7259


In [5]:
# --- INFERENCE DEMO ---

# pick a random sample to test inference logic
sample_idx = 0
test_seq = X_train[sample_idx]

# remove the padding (-999) to simulate real-world input of unknown length
real_len = np.sum(~np.all(test_seq == -999, axis=1))
clean_seq = test_seq[:real_len]

print(f"Testing on sequence length: {real_len} years")

probs = shutdown_model.predict_status(clean_seq)

print("\nYearly shutdown probabilities:")
print(np.round(probs, 2))

# threshold check
threshold = 0.5
shutdown_indices = np.where(probs > threshold)[0]

if len(shutdown_indices) > 0:
    # +1 for human readable year (1-indexed)
    print(f"\nPREDICTION: Resort shuts down at Year {shutdown_indices[0] + 1}")
else:
    print("\nPREDICTION: Resort survives (>15 years)")

Testing on sequence length: 11 years



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step



Yearly shutdown probabilities:
[0.41 0.26 0.21 0.19 0.17 0.14 0.15 0.16 0.18 0.24 0.28]

PREDICTION: Resort survives (>15 years)
